<a href="https://colab.research.google.com/github/SeongcheolKim/word2vec/blob/main/coco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
#데이터 로드
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [5]:
#이미지 크기 변경 및 정규화(32, 32)->(224, 224)
#배치 크기 설정
batch_size = 32

# 레이블 원-핫 인코딩
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

#데이터셋을 만들고 즉시 리사이징하지 않음 (메모리 절약)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

#배치 단위로 변환 (메모리 사용 최적화)
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))  # 실시간 리사이징 (RAM 사용 감소)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

#`map`을 사용해 데이터 변환 (메모리 최적화)
train_dataset = train_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

#배치 단위로 변환 (num_parallel_calls 최적화)
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [6]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False

In [7]:
#fine-tuning 용 FC 추가
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation = 'relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation = 'softmax')
])

In [8]:
model.compile(
    optimizer = keras.optimizers.RMSprop(learning_rate=1e-05),
    loss = "categorical_crossentropy",
    metrics = ["accuracy"]
)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = "model_feature.keras",
        save_best_only = True,
        monitor = "val_loss"
    )
]

history = model.fit(
    train_dataset,
    epochs = 10,
    validation_data = test_dataset,
    callbacks = callbacks
)

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 174s 103ms/step - accuracy: 0.0993 - loss: 2.5796 - val_accuracy: 0.1351 - val_loss: 2.2969
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 151s 96ms/step - accuracy: 0.1069 - loss: 2.3306 - val_accuracy: 0.1797 - val_loss: 2.2913
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 151s 96ms/step - accuracy: 0.1185 - loss: 2.2962 - val_accuracy: 0.2011 - val_loss: 2.2868
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 151s 96ms/step - accuracy: 0.1421 - loss: 2.2867 - val_accuracy: 0.2004 - val_loss: 2.2820
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 151s 97ms/step - accuracy: 0.1542 - loss: 2.2827 - val_accuracy: 0.2047 - val_loss: 2.2778
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 151s 97ms/step - accuracy: 0.1617 - loss: 2.2776 - val_accuracy: 0.2337 - val_loss: 2.2734
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 151s 96ms/step - accuracy: 0.1641 - loss: 2.2736 - val_accuracy: 0.2082 - val_loss: 2.2694
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 151s 96ms/step - accuracy:

In [10]:
# ResNet50의 마지막 몇 개의 레이어를 학습 가능하게 변경
for layer in base_model.layers[-50:]:  # 마지막 50개 레이어만 Fine-Tuning
    layer.trainable = True

# 모델 재컴파일 (낮은 Learning Rate 사용)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),  # 낮은 LR로 미세 조정
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = "model_feature.keras",
        save_best_only = True,
        monitor = "val_loss"
    )
]

# Fine-Tuning 학습
history_finetune = model.fit(
    train_dataset,
    epochs=5,  # 추가 5 Epochs
    validation_data=test_dataset,
    callbacks=callbacks
)

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 282s 165ms/step - accuracy: 0.3336 - loss: 1.8388 - val_accuracy: 0.3791 - val_loss: 1.7165
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 242s 155ms/step - accuracy: 0.4883 - loss: 1.4391 - val_accuracy: 0.4007 - val_loss: 2.0210
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 242s 155ms/step - accuracy: 0.5301 - loss: 1.3353 - val_accuracy: 0.3508 - val_loss: 2.1804
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 242s 155ms/step - accuracy: 0.5508 - loss: 1.2798 - val_accuracy: 0.4154 - val_loss: 1.7217
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 241s 154ms/step - accuracy: 0.5672 - loss: 1.2371 - val_accuracy: 0.3839 - val_loss: 1.9277
